
# EDA2 — Complete Notebook

**Objective:** Complete EDA, preprocessing, encoding, feature engineering, and feature selection for the Adult dataset (predicting income >50K).

This notebook was generated automatically to satisfy the four tasks specified in `EDA2.docx`.

**Usage notes:**
- If you run this on your local machine (e.g., `D:\DATA-SCIENCE\ASSIGNMENTS\12 EDA2`), set `DATA_PATH` below to the local CSV path. The notebook will try the local path first and fall back to `/mnt/data/adult_with_headers.csv` if not found.
- Some optional libraries (like `ppscore`) may not be available in offline environments. In that case, a Mutual Information fallback is used for approximate predictive power analysis.

---


In [1]:

# 1) Data loading
import pandas as pd, os
# Set DATA_PATH to your local CSV location if running locally. Example:
# DATA_PATH = r"D:\DATA-SCIENCE\ASSIGNMENTS\12 EDA2\adult_with_headers.csv"
# Otherwise leave it None to use the sandbox copy.
DATA_PATH = None

possible_paths = [
    DATA_PATH,
    r"D:\DATA-SCIENCE\ASSIGNMENTS\12 EDA2\adult_with_headers.csv",
    "/mnt/data/adult_with_headers.csv",
    "/mnt/data/adult_with_headers (1).csv",
    "/mnt/data/adult_with_headers.csv"
]

csv_path = None
for p in possible_paths:
    if p and os.path.exists(p):
        csv_path = p
        break

if csv_path is None:
    raise FileNotFoundError("adult_with_headers.csv not found. Place it in the working directory or change DATA_PATH variable. Tried: " + ", ".join([str(p) for p in possible_paths]))

print("Loading dataset from:", csv_path)
df = pd.read_csv(csv_path)
print("Shape:", df.shape)
display(df.head(6))


Loading dataset from: D:\DATA-SCIENCE\ASSIGNMENTS\12 EDA2\adult_with_headers.csv
Shape: (32561, 15)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [2]:

# 2) Basic EDA
print("Info:")
display(df.info())
print("\n---\nDescribe (numeric):")
display(df.describe().T)
print("\n---\nCategorical columns and value counts (top 5 each):")
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
for c in cat_cols:
    print(f"\nColumn: {c} (nunique={df[c].nunique()})")
    display(df[c].value_counts(dropna=False).head(5))
    
print("\nMissing values per column:")
display(df.isin(['?', ' ?']).sum().rename("count_of_question_mark").to_frame())
display(df.isnull().sum().to_frame())


Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


None


---
Describe (numeric):


,count,mean,std,min,25%,50%,75%,max
age,32561.0,38.581647,13.640433,17.0,28.0,37.0,48.0,90.0
fnlwgt,32561.0,189778.366512,105549.977697,12285.0,117827.0,178356.0,237051.0,1484705.0
education_num,32561.0,10.080679,2.572720,1.0,9.0,10.0,12.0,16.0
capital_gain,32561.0,1077.648844,7385.292085,0.0,0.0,0.0,0.0,99999.0
capital_loss,32561.0,87.303830,402.960219,0.0,0.0,0.0,0.0,4356.0
hours_per_week,32561.0,40.437456,12.347429,1.0,40.0,40.0,45.0,99.0



---
Categorical columns and value counts (top 5 each):

Column: workclass (nunique=9)


...


Column: income (nunique=2)


income
<=50K    24720
>50K      7841
Name: count, dtype: int64


Missing values per column:


,count_of_question_mark
age,0
workclass,1836
fnlwgt,0
education,0
education_num,0
marital_status,0
occupation,1843
relationship,0
race,0
sex,0


,0
age,0
workclass,0
fnlwgt,0
education,0
education_num,0
marital_status,0
occupation,0
relationship,0
race,0
sex,0


In [3]:

# 3) Missing value handling / Cleaning
import numpy as np
# Many Adult dataset variants use '?' to indicate missing values.
df_clean = df.copy()
for c in df_clean.columns:
    if df_clean[c].dtype == object:
        df_clean[c] = df_clean[c].replace(' ?', np.nan).replace('?', np.nan).str.strip()

# Show counts again
print("Missing values after cleaning '?' -> NaN:")
display(df_clean.isnull().sum().sort_values(ascending=False).to_frame("missing_count"))

# Simple strategy: for categorical columns, fill missing with 'Missing'; for numeric, fill with median.
cat_cols = df_clean.select_dtypes(include=['object','category']).columns.tolist()
num_cols = df_clean.select_dtypes(include=['number']).columns.tolist()

for c in cat_cols:
    if df_clean[c].isnull().sum() > 0:
        df_clean[c] = df_clean[c].fillna("Missing")

for c in num_cols:
    if df_clean[c].isnull().sum() > 0:
        df_clean[c] = df_clean[c].fillna(df_clean[c].median())

print("After imputation, missing counts:")
display(df_clean.isnull().sum().sum())


Missing values after cleaning '?' -> NaN:


,missing_count
occupation,1843
workclass,1836
native_country,583
age,0
fnlwgt,0
education,0
education_num,0
marital_status,0
relationship,0
race,0


After imputation, missing counts:


np.int64(0)

In [5]:
# Save scalers for later use (if needed)
import pickle
import os

save_dir = r"D:\DATA-SCIENCE\ASSIGNMENTS\12 EDA2"
os.makedirs(save_dir, exist_ok=True)

with open(os.path.join(save_dir, "standard_scaler.pkl"), 'wb') as f:
    pickle.dump(sc, f)

with open(os.path.join(save_dir, "minmax_scaler.pkl"), 'wb') as f:
    pickle.dump(mms, f)

print(f"✅ Scalers saved successfully to {save_dir}")


✅ Scalers saved successfully to D:\DATA-SCIENCE\ASSIGNMENTS\12 EDA2


In [7]:

# 4) Scaling examples: StandardScaler and MinMaxScaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaled_examples = df_clean.copy()

# Apply scaling to numeric columns; keep copies for comparison
num_cols = scaled_examples.select_dtypes(include=['number']).columns.tolist()
print("Numeric columns to scale:", num_cols)

sc = StandardScaler()
mms = MinMaxScaler()

scaled_standard = scaled_examples.copy()
scaled_minmax = scaled_examples.copy()

scaled_standard[num_cols] = sc.fit_transform(scaled_standard[num_cols])
scaled_minmax[num_cols] = mms.fit_transform(scaled_minmax[num_cols])

print("Standard scaled (showing head):")
display(scaled_standard[num_cols].head())
print("MinMax scaled (showing head):")
display(scaled_minmax[num_cols].head())

# Save scalers for later use (if needed)
save_dir = r"D:\DATA-SCIENCE\ASSIGNMENTS\12 EDA2"
os.makedirs(save_dir, exist_ok=True)

with open(os.path.join(save_dir, "standard_scaler.pkl"), 'wb') as f:
    pickle.dump(sc, f)

with open(os.path.join(save_dir, "minmax_scaler.pkl"), 'wb') as f:
    pickle.dump(mms, f)

print(f"✅ Scalers saved successfully to {save_dir}")

Numeric columns to scale: ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']
Standard scaled (showing head):


,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
0,0.030671,-1.063611,1.134739,0.148453,-0.21666,-0.035429
1,0.837109,-1.008707,1.134739,-0.145920,-0.21666,-2.222153
2,-0.042642,0.245079,-0.420060,-0.145920,-0.21666,-0.035429
3,1.057047,0.425801,-1.197459,-0.145920,-0.21666,-0.035429
4,-0.775768,1.408176,1.134739,-0.145920,-0.21666,-0.035429


MinMax scaled (showing head):


,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
0,0.301370,0.044302,0.800000,0.02174,0.0,0.397959
1,0.452055,0.048238,0.800000,0.00000,0.0,0.122449
2,0.287671,0.138113,0.533333,0.00000,0.0,0.397959
3,0.493151,0.151068,0.400000,0.00000,0.0,0.397959
4,0.150685,0.221488,0.800000,0.00000,0.0,0.397959


✅ Scalers saved successfully to D:\DATA-SCIENCE\ASSIGNMENTS\12 EDA2


In [8]:

# 5) Encoding categorical variables
from sklearn.preprocessing import LabelEncoder
encoded = df_clean.copy()

cat_cols = encoded.select_dtypes(include=['object','category']).columns.tolist()
unique_counts = encoded[cat_cols].nunique().sort_values()

onehot_cols = unique_counts[unique_counts < 5].index.tolist()
label_cols = unique_counts[unique_counts >= 5].index.tolist()

print("One-hot cols (<5):", onehot_cols)
print("Label cols (>=5):", label_cols)

# One-Hot Encoding for low-cardinality categoricals
if onehot_cols:
    encoded = pd.get_dummies(encoded, columns=onehot_cols, drop_first=False, prefix_sep='__')

# Label Encoding for higher-cardinality categoricals
le = LabelEncoder()
applied_label_cols = []
for col in label_cols:
    # Fill missing placeholders already done
    encoded[col] = le.fit_transform(encoded[col].astype(str))
    applied_label_cols.append(col)

print("Encoded dataframe shape:", encoded.shape)
display(encoded.head(6))

# Save intermediate encoded csv
encoded.to_csv("adult_encoded_intermediate.csv", index=False)
print("Saved encoded intermediate CSV to adult_encoded_intermediate.csv (in current working dir).")


One-hot cols (<5): ['sex', 'income']
Label cols (>=5): ['race', 'relationship', 'marital_status', 'workclass', 'occupation', 'education', 'native_country']
Encoded dataframe shape: (32561, 17)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country,sex__Female,sex__Male,income__<=50K,income__>50K
0,39,7,77516,9,13,4,0,1,4,2174,0,40,39,False,True,True,False
1,50,6,83311,9,13,2,3,0,4,0,0,13,39,False,True,True,False
2,38,4,215646,11,9,0,5,1,4,0,0,40,39,False,True,True,False
3,53,4,234721,1,7,2,5,0,2,0,0,40,39,False,True,True,False
4,28,4,338409,9,13,2,10,5,2,0,0,40,4,True,False,True,False
5,37,4,284582,12,14,2,3,5,4,0,0,40,39,True,False,True,False


Saved encoded intermediate CSV to adult_encoded_intermediate.csv (in current working dir).


In [9]:

# 6) Feature Engineering
fe = encoded.copy()

# Example new features — create only if relevant columns exist
# 1) education_num_per_age = education-num / age (if both exist)
if 'education-num' in fe.columns and 'age' in fe.columns:
    fe['edu_per_age'] = fe['education-num'] / (fe['age'].replace(0, 1))
    print("Created feature: edu_per_age")

# 2) hours_per_week_per_occupation_freq: if hours-per-week and occupation exist (occupation may have been label-encoded)
if 'hours-per-week' in fe.columns:
    # create a simple interaction: hours_per_week / (1 + number of dependents proxy)
    # We don't have dependents; create hours_per_week_norm = hours-per-week / age as a proxy
    if 'age' in fe.columns:
        fe['hours_per_age'] = fe['hours-per-week'] / (fe['age'].replace(0, 1))
        print("Created feature: hours_per_age")

# 3) Log transform of a skewed numeric feature (capital-gain if present)
import numpy as np
if 'capital-gain' in fe.columns:
    fe['capital_gain_log1p'] = np.log1p(fe['capital-gain'].clip(lower=0))
    print("Created feature: capital_gain_log1p (log1p of capital-gain)")

print("New features added. Showing head:")
display(fe.head(6))


New features added. Showing head:


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country,sex__Female,sex__Male,income__<=50K,income__>50K
0,39,7,77516,9,13,4,0,1,4,2174,0,40,39,False,True,True,False
1,50,6,83311,9,13,2,3,0,4,0,0,13,39,False,True,True,False
2,38,4,215646,11,9,0,5,1,4,0,0,40,39,False,True,True,False
3,53,4,234721,1,7,2,5,0,2,0,0,40,39,False,True,True,False
4,28,4,338409,9,13,2,10,5,2,0,0,40,4,True,False,True,False
5,37,4,284582,12,14,2,3,5,4,0,0,40,39,True,False,True,False


In [10]:

# 7) Outlier detection with IsolationForest
from sklearn.ensemble import IsolationForest
import numpy as np

numeric_cols = fe.select_dtypes(include=['number']).columns.tolist()
print("Numeric columns used for isolation forest:", numeric_cols)

# Fit IsolationForest on numeric columns (use a smaller random_state for reproducibility)
iso = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
iso.fit(fe[numeric_cols])

# Predict: -1 for outliers, 1 for inliers
preds = iso.predict(fe[numeric_cols])
fe['iso_outlier'] = preds
print("Outlier counts:", (preds == -1).sum())

# Remove outliers and save
fe_no_outliers = fe[fe['iso_outlier'] == 1].copy()
print("Shape before removing outliers:", fe.shape)
print("Shape after removing outliers:", fe_no_outliers.shape)
fe_no_outliers.to_csv("adult_no_outliers.csv", index=False)
print("Saved adult_no_outliers.csv to current working directory.")


Numeric columns used for isolation forest: ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country']
Outlier counts: 326
Shape before removing outliers: (32561, 18)
Shape after removing outliers: (32235, 18)
Saved adult_no_outliers.csv to current working directory.


In [12]:
# Robust PPS / MI cell (replacement)
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression

# Try PPS first
try:
    import ppscore as pps
    print("ppscore found — computing PPS matrix (may take a while)...")
    pps_matrix = pps.matrix(fe_no_outliers).pivot(index='x', columns='y', values='ppscore')
    display(pps_matrix)
    pps_matrix.to_csv(r"D:\DATA-SCIENCE\ASSIGNMENTS\12 EDA2\pps_matrix.csv", index=True)
    print("Saved pps_matrix.csv to your local folder.")
except Exception as e:
    print("ppscore not available or failed. Falling back to Mutual Information.")
    print("ppscore error:", e)
    # --- Find a target column robustly ---
    # Candidate names (common)
    candidate_names = [c for c in fe_no_outliers.columns if c.lower() in ['income','target','label','y','class']]
    if len(candidate_names) == 0:
        # check original cleaned df for typical target names
        candidate_names = [c for c in df_clean.columns if 'income' in c.lower() or 'target' in c.lower()]
    if len(candidate_names) == 0:
        # If still none, try to detect a binary/object column in df_clean that looks like target
        possible = [c for c in df_clean.select_dtypes(include=['object','category']).columns
                    if df_clean[c].nunique() <= 5]
        # Heuristic: income-like values contain '>' or '50K' etc
        heur = [c for c in possible if df_clean[c].astype(str).str.contains('50K|>50K|<=50K|> 50K|<= 50K|high income', case=False, na=False).any()]
        if len(heur)>0:
            candidate_names = heur
    if len(candidate_names) == 0:
        print("No obvious target found automatically. Please set `target_col` manually to the name of your label column.")
        target_col = None
    else:
        target_col = candidate_names[0]
        print("Auto-detected target candidate:", target_col)
    
    # If target detected but not present in fe_no_outliers (e.g., it was one-hoted or label-encoded earlier),
    # try to rebuild y from df_clean using index alignment
    y = None
    if target_col is not None:
        if target_col in fe_no_outliers.columns:
            y = fe_no_outliers[target_col]
            print("Using target from fe_no_outliers.")
        elif target_col in df_clean.columns:
            # align indices: fe_no_outliers may be a subset with same original indices
            try:
                # If fe_no_outliers preserved original indices, use them
                y = df_clean.loc[fe_no_outliers.index, target_col]
                print("Reconstructed target y by aligning df_clean -> fe_no_outliers via index.")
            except Exception:
                # fallback: use the original whole column (not ideal if rows were removed)
                y = df_clean[target_col]
                print("Used full column from df_clean (note: lengths may not match if rows were removed).")
        else:
            # maybe target was one-hoted. Check for columns starting with target_col + delimiter
            possible_onehot = [c for c in fe_no_outliers.columns if c.startswith(str(target_col) + '__') or c.startswith(str(target_col) + '_')]
            if len(possible_onehot)==1:
                # that column is probably the target indicator
                y = fe_no_outliers[possible_onehot[0]]
                print("Found one-hot style target column:", possible_onehot[0])
            elif len(possible_onehot)>1:
                print("Multiple one-hot columns found for target name; please identify which column represents the label.")
                y = None

    # If still no y found, ask user to set manually (but per instructions we'll not block; just print)
    if y is None:
        print("Could not automatically construct target (y). If you know the target column name, set `target_col = 'your_column'` and rerun this cell.")
    else:
        # ensure y is numeric labels for mutual info
        if y.dtype == object or y.dtype.name == 'category':
            y = y.astype('category').cat.codes

        # Build X: numeric features only (drop iso_outlier if present)
        X = fe_no_outliers.select_dtypes(include=['number']).drop(columns=['iso_outlier'], errors='ignore').fillna(0)
        # Ensure X and y align in length
        if len(X) != len(y):
            # try to reindex y to X
            try:
                y = y.reindex(X.index)
                print("Aligned y to X by reindexing.")
            except Exception as e:
                print("Could not align X and y; aborting MI. Error:", e)
                X = None

        if X is not None and len(X)==len(y):
            # choose MI function based on y cardinality
            if y.nunique() <= 10:
                mi = mutual_info_classif(X, y, discrete_features='auto', random_state=42)
            else:
                mi = mutual_info_regression(X, y)
            mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
            display(mi_series.head(50).to_frame("mutual_info_score"))
            outpath = r"D:\DATA-SCIENCE\ASSIGNMENTS\12 EDA2\mutual_info_scores.csv"
            mi_series.to_csv(outpath)
            print("Saved mutual_info_scores.csv to:", outpath)
        else:
            print("X or y not properly prepared — mutual information not computed.")


ppscore not available or failed. Falling back to Mutual Information.
ppscore error: No module named 'ppscore'
Auto-detected target candidate: income
Reconstructed target y by aligning df_clean -> fe_no_outliers via index.


,mutual_info_score
relationship,0.117282
marital_status,0.113404
capital_gain,0.081693
age,0.070203
occupation,0.066159
education,0.064986
education_num,0.064661
hours_per_week,0.038364
capital_loss,0.034617
fnlwgt,0.033343


Saved mutual_info_scores.csv to: D:\DATA-SCIENCE\ASSIGNMENTS\12 EDA2\mutual_info_scores.csv



## Comparison: Correlation Matrix vs PPS / Mutual Information

- The correlation matrix shows **linear** relationships; PPS and Mutual Information can capture non-linear predictive relationships.
- If a feature has low linear correlation but high PPS/MI, it may still be useful for non-linear models.
- Use both tools: correlation for initial multicollinearity checks and PPS/MI for predictive importance.



In [14]:
# ✅ FINAL SAVES — LOCAL VERSION
# All outputs will be saved directly into your working directory
import os
import shutil

# Define your working directory
save_dir = r"D:\DATA-SCIENCE\ASSIGNMENTS\12 EDA2"

# Ensure directory exists
os.makedirs(save_dir, exist_ok=True)

# Move any intermediate files that exist in current dir
for fname in ['adult_encoded_intermediate.csv', 'adult_no_outliers.csv']:
    if os.path.exists(fname):
        shutil.move(fname, os.path.join(save_dir, fname))
        print(f"Moved {fname} → {save_dir}")

# Save the final processed datasets
try:
    fe.to_csv(os.path.join(save_dir, "adult_encoded_full_pipeline.csv"), index=False)
    fe_no_outliers.to_csv(os.path.join(save_dir, "adult_no_outliers.csv"), index=False)
    print(f"✅ Saved final encoded dataset → {save_dir}\\adult_encoded_full_pipeline.csv")
    print(f"✅ Saved outlier-removed dataset → {save_dir}\\adult_no_outliers.csv")
except Exception as e:
    print("⚠️ Could not save automatically. Error:", e)

print("\n🎉 Notebook complete! All files saved in your working directory:")
print(save_dir)


Moved adult_encoded_intermediate.csv → D:\DATA-SCIENCE\ASSIGNMENTS\12 EDA2
Moved adult_no_outliers.csv → D:\DATA-SCIENCE\ASSIGNMENTS\12 EDA2
✅ Saved final encoded dataset → D:\DATA-SCIENCE\ASSIGNMENTS\12 EDA2\adult_encoded_full_pipeline.csv
✅ Saved outlier-removed dataset → D:\DATA-SCIENCE\ASSIGNMENTS\12 EDA2\adult_no_outliers.csv

🎉 Notebook complete! All files saved in your working directory:
D:\DATA-SCIENCE\ASSIGNMENTS\12 EDA2
